# Part 1: GP Regression

In the first part of the tutorial, we are going to use the GPy library for Gaussian process regression (https://github.com/SheffieldML/GPy). GPy is available via `pip install GPy`.

In [ ]:
import GPy
import numpy as np
import matplotlib.pyplot as plt

As a simple benchmark problem, we use a small mockup machine interface.

In [ ]:
class Accelerator:
    def __init__(self):
        self.x = np.array([0.1, 0.2])
    
    def get(self, **kwargs):
        """
        Returns the signal measurement.
        
        Examples:
            get()  # returns a measurement with the current parameter settings
            get(x=[0.1,0.4])  # sets the parameters x1=0.1, x2=0.4, then returns a measurement
            get(x1=0.2)  # sets x1=0.2 and keeps the second parameter fixed
            get(x1=0.1, x2=0.4)  # sets x1=0.1, x2=0.4
        
        Optional parameters:
            x: array-like 2d
            x1: scalar
            x2: scalar
        """
        if 'x' in kwargs:
            self.x = np.array(kwargs['x'])
        else:
            for i in range(len(self.x)):
                self.x[i] = kwargs.get(f'x{i+1}', self.x[i])
        
        xx, yy = self.x[0], self.x[1]
        y = (4. - 2.1*xx**2 + (xx**4)/3.)*(xx**2) + xx*yy + (-4. + 4*(yy**2))*(yy**2)
        return np.maximum(-y + 2.5, 0) + np.random.normal(0,0.3)

The machine interface maintains a parameter vector `accelerator.x`. With a call to `accelerator.get()`, the objective signal can be measured. You can also evaluate at a new parameter using the keyword arguments `accelerator.get(x=[0.1,0.2])` or `accelerator.get(x1=0.1, x2=0.2)`.

In [ ]:
# initialize mockup machine interface
accelerator = Accelerator()

# retrieving current parameter and (noisy) signal
print(f"The current parameter settings are: {accelerator.x}")
print(f"Signal measurement: {accelerator.get():.2f}")
print(f"Another measurement: {accelerator.get():.2f}")

# set x1 and x2 parameters and new measurement of signal:
print(f"Setting x1=0.4: New measurement: {accelerator.get(x1=0.4):.2f} at x={accelerator.x}")
print(f"Setting x2=2: New measurement: {accelerator.get(x2=2):.2f} at x={accelerator.x}")

# set both parameters
print(f"Setting x=[0.5,0.5]: New measurement: {accelerator.get(x=[0.5,0.5]):.2f} at x={accelerator.x}")

Below we define some helper functions for plotting GPs. You can come back to these definitions later to understand the details of how to query the GP predictions. For now, just run the cell and go on to the next section.

In [ ]:
def plot_1d_gp(gp, xlim, 
               plot_data=True, 
               plot_confidence=False, 
               s=2,
               axis=None, color='C0'):
    """
    Plots a 1d gp.
    """
    
    if axis is None:
        axis = plt.gca()
        
    # define an evaluation set
    X_eval = np.linspace(*xlim, 300).reshape(-1,1)  # GPy expects shape (num_points, input_dim)
    
    # compute posterior mean and variance
    Y_pred, Y_var = gp.predict_noiseless(X_eval)
    
    # flatten the arrays for pyplot
    X_eval = X_eval.reshape(-1)
    Y_pred = Y_pred.reshape(-1)
    Y_var = Y_var.reshape(-1)

    # plot mean
    axis.plot(X_eval, Y_pred, zorder=5, color=color)
    
    # plot data if it's not the 'fake' data point
    if plot_data and not (gp.X.shape[0] == 1 and gp.X[0] > 0.9*10e10):
        axis.scatter(gp.X, gp.Y, zorder=10, marker='x', color=color)
        
    # plot the predictive uncertainty interval
    if plot_confidence:
        axis.fill_between(X_eval, Y_pred-s*np.sqrt(Y_var), Y_pred+s*np.sqrt(Y_var), alpha=0.4, color=color)

    axis.set_xlim(xlim)  # set x-axis limits


def plot_1d_gp_samples(gp, xlim, num_samples=5, plot_data=True, axis=None, color_samples='C1'):
    """
    Plots `num_samples` samples of a gp.
    """
    
    if axis is None:
        axis = plt.gca()
        
    X_eval = np.linspace(*xlim, 300).reshape(-1,1)  # GPy expects shape (num_points, input_dim)

    # Genereate posterior samples
    Y_sample = gp.posterior_samples_f(X_eval, size=num_samples)

    # plotting
    for i in range(Y_sample.shape[-1]):
        axis.plot(X_eval, Y_sample[:,:,i], c=color_samples)
    
    # plot data if it's not the 'fake' data point
    if plot_data and not (gp.X.shape[0] == 1 and gp.X[0] > 0.9*10e10):
        axis.scatter(gp.X, gp.Y, marker='x', zorder=10, c='C0')
        
    axis.set_xlim(xlim)  # set x-axis limits

## Generating a data set and fitting a GP

Here we are going to evaluate our accelerator on a grid to generate a data set. We then use a GP to regress the target function.

As you go on you can get back to the next cell and change the number of evaluation points to obtain a different sized data-set. Unfortunately, GPy does not allow an empty data set; to effectively get the prior distribution, we can however add a single data point far away from our domain of interest (set `n=0`).

Once we have the data `X` and `Y`, we can compute the posterior GP using GPy. To do so, we first define an RBF kernel, and them get a `GPy.models.GPRegression` instance with our data and the chosen kernel. 

*Note that GPy always expects numpy arrays with shape (num_data_points, input_dim) for X and (num_data_points, 1) for Y as input.*

#### Things to try:
* Rerun the cell to obtain a different (noisy) measurement.
* To change the number of evaluation points, set `n` to a different value or `n=0` to get th GP prior distribution.

In [ ]:
# number of evaluations
n = 20

if n > 0:
    # evaluation points
    X = np.linspace(-2,2,n).reshape(n,1)

    # call the machine interface to obtain observations
    Y = np.array([accelerator.get(x1=x) for x in X]).reshape(n,1)

    # quick scatter plot
    plt.scatter(X,Y, marker='x')
    plt.title('Observations')

else:
    # simulate the prior distribution by adding a point 
    # far away from the domain of interest.
    # This is necessary because GPy does not allow an empty data set.
    X, Y = np.array([[10e10]]), np.array([[0.]])

# Define a kernel. We use RBF (or squared exponential to start with)
rbf = GPy.kern.RBF(input_dim=1, lengthscale=1, variance=10)
# Define the GP object
gp = GPy.models.GPRegression(X=X, Y=Y, kernel=rbf, noise_var=0.1)

### Mean and variance predictions

The posterior mean and variance can be obtain via `y,v = gp.predict_noiseless(X1)` for some evaluation point `X1` (we only change a single parameter here). If you are interested in including the noise in the predictive variance, you can use `y,v = gp.predict(X1)` instead. For Bayesian optimization, we are primarly intersted in the former, as our goal is to maximize the expected return value $f(x)$ of the target function.

#### Things to try:
* Change the evaluation point X1
* use `.predict(X1)` instead of `predict_noiseless(X1)` to include the noise variance.
* Increase the number of data points or use the prior distribution

In [ ]:
X1 = np.array([[0.2]])  # need shape=(1,1)

# For BO we are typically interested in the uncertainty in the posterior mean (f(x))
# Since this distribution is Gaussian, it is specified by its mean and variance,
# which we obtain form the following call:
Y1, V1 = gp.predict_noiseless(X1)

# to include the noise varianc, use 
# Y1, V1 = gp.predict(X1)  

print(f"Posterior at x = {float(X1)}:  y = {float(Y1):.2} ± {float(np.sqrt(V1)):.2}")

To plot mean and variance of our GP model, we have to first define a grid of evaluation points `X` and then call `Y,V = gp.predict_noiseless(X)` to obtain the preditive mean and variance. Then, we can compute predictive intervals via `Y - np.sqrt(V),  Y + np.sqrt(V)`. All of this is handled in the helper function `plot_1d_gp`, which was defined above. Have a look there how the plots are generated: Besides the plotting it uses nothing more than what was introduced in the previous cell.


#### Things to try:
* Change the evaluation set with `xlim`.
* Change the number of data points (going back to the data-set definition above)
* Go to the definition of `plot_1d_gp` above to see how we obtain the predictive intervals from the GP

In [ ]:
plot_1d_gp(gp, xlim=(-8,5), plot_confidence=True)

### Sampling from the GP

We can obtain a posterior sample from our GP by calling `gp.posterior_samples_f(X_eval, size=1)` evaluated on points `X_eval`. Plotting functionality is provided via `plot_1d_gp_samples` as defined above.

#### Things to try:

* Re-run the cell to obtain different samples
* Sample from the prior distribution
* Add more data points
* Go to the definition of `plot_1d_gp_samples` to see how we can obtain posterior samples from the gp
* Generate the samples directly from the posterior mean and covariance (cell below).

In [ ]:
plot_1d_gp_samples(gp, xlim=(-4,4), num_samples=5)
plt.title('GP Samples')

#### Intermezzo: Generating GP samples
Recall that a $GP(m,K)$ is a distribution of functions f such that every finite marginal $f(x_1), ..., f(x_n)$ is a multivariate Gaussian with mean $[m(x_1), ..., m(x_n)]$ and covariance $[Cov(x_i, x_j)] = [K(x_i,x_j)]$.

This can be used to generate the sample paths manually, and that is exactly what is behind `.posterior_samples_f(...)`.



In [ ]:
# Step 1: Define evaluation grid
X_eval = np.linspace(-4,4,100).reshape(-1,1)

# Step 2: Compute posterior mean and covariance matrix
m, K = gp.predict_noiseless(X_eval, full_cov=True)

# Step 3: Sample a multi-variate normal N(m,K)
Y_sample = np.random.multivariate_normal(m.flatten(), K)

# plotting
plt.plot(X_eval.flatten(), Y_sample, color='C3')

### Exploring the Hyperparameters

Go back and generate a data set with `n=20` points. We will now explore what happens if we change the GP hyper-parameters.

#### Things to try:
* Change the lengthscale. Observe how you start overfitting with too small lengthscale, and underfitting with too large lengthscale.
* Change the variance: This is the range of observation values we expect.
* Change the noise variance: The larger this value, the more robust the regression is against pertubations in the observations.
* Can you explain what happens if we set the noise variance to a very small value?

In [ ]:
gp.kern.lengthscale = 1  # lengthscale, e.g. 0.1, 1., 10.
gp.kern.variance = 10  # prior variance 1, 10, 100
gp.likelihood.variance = 0.1  # noise variance e.g. 1, 0.1, 0.01, 0.0001, 0.00001.

_, (axis1, axis2) = plt.subplots(1, 2, sharey=True, figsize=(10,3.5))

# plot model uncertainty on axis1
axis1.set_title('Model uncertainty')
plot_1d_gp(gp, xlim=(-4,4), 
           plot_data=True,
           plot_confidence=True,
           axis=axis1)

# plot samples on axis2
axis2.set_title('Posterior Samples')
plot_1d_gp_samples(gp, num_samples=5, xlim=(-4,4),
                   plot_data=True,
                   axis=axis2)

### Optimize Hyperparameters

GPy also offers a way of optimizing the hyper-parameters via the negative log-likelihood. Simply call `gp.optimize()`, or better `gp.optimize_restarts()`. The latter uses multiple restarts to escape local minima, as the likelihood is non-convex in general. Be careful however that for a good point-estiamte, you will need representative prior data of your optimization problem. Optimizing the paramters with data collected from the optimization often does not work very well.

#### Things to try
* Use a very small number (n=2) or a large number (n=100) of data points.
* Do the parameters found agree with the 'good' settings found above?

In [ ]:
gp.optimize_restarts()  # optimizes parameters via log-likelihood

display(gp)  # print resulting gp parameters


# plotting, as before 
_, (axis1, axis2) = plt.subplots(1, 2, sharey=True, figsize=(10,3.5))

# plot model uncertainty on axis1
axis1.set_title('Model uncertainty')
plot_1d_gp(gp, xlim=(-4,4), 
           plot_data=True,
           plot_confidence=True,
           axis=axis1)

# plot samples on axis2
axis2.set_title('Posterior Samples')
plot_1d_gp_samples(gp, num_samples=5, xlim=(-4,4),
                   plot_data=True,
                   axis=axis2)

### Changing the Kernel

So far we have been using the RBF (or squared exponential) kernel. This is a common starting point for exploring the GP regression. However, depending on the function we want to model, choosing a different kernel can make a lot of sense. 

#### Things to try:
* Uncomment the different kernels below and see how the regression and sample plots look like
* You can also vary GP hyperparameter as before, 
* Uncomment the call to `.optimize_restarts()` to automatically adjust the hyperparameters.

In [ ]:
kernel = GPy.kern.Bias(input_dim=1)  # a constant kernel
# kernel = GPy.kern.Linear(input_dim=1) 

# kernel = GPy.kern.Bias(input_dim=1) + GPy.kern.Linear(input_dim=1)
# kernel = GPy.kern.Poly(input_dim=1, order=2)
# kernel = GPy.kern.Matern32(input_dim=1, lengthscale=1., variance=10)
# kernel = GPy.kern.Exponential(input_dim=1, lengthscale=1., variance=10)

# kernel = GPy.kern.Matern52(input_dim=1, lengthscale=0.5, variance=10)
# kernel = GPy.kern.RBF(input_dim=1, lengthscale=0.5, variance=10)

# Combine different kernels
# kernel = GPy.kern.Bias(input_dim=1) \
#             + GPy.kern.Matern32(input_dim=1, lengthscale=0.5) \
#             + GPy.kern.RBF(input_dim=1, lengthscale=0.5) 

gp = GPy.models.GPRegression(X=X, Y=Y, kernel=kernel, noise_var=0.1)

# uncomment the next line if you want to optimize the hyperparamters
# gp.optimize_restarts()  # optimizes parameters via log-likelihood

display(gp)  # print resulting gp parameters

fig, (axis1, axis2) = plt.subplots(1, 2, figsize=(10,3.5))


axis1.set_title("Model Uncertainty")
plot_1d_gp(gp, xlim=(-4,4), 
           plot_data=True,
           plot_confidence=True,
           axis=axis1, s=1)

axis2.set_title("Posterior Samples")
plot_1d_gp_samples(gp, xlim=(-4,4),
                   plot_data=True,
                   axis=axis2)

## Gaussian Regression in Higher Dimensions

GP regression works the same in higher dimensions. We start by defining 2d plotting helpers. 

In [ ]:
def plot_gp_contour(gp, boundaries, axis=None, fig=None):
    """
    Helper function for a 2d contour plot of a GP.
    """
    
    if fig is None:
        fig = plt.gcf()
    if axis is None:
        axis = fig.gca()
    
    # plot data
    axis.scatter(gp.X[:,0], gp.X[:,1], marker='x', zorder=10)
    
    # create evaluation grid
    x = np.linspace(*boundaries[0], 100)
    y = np.linspace(*boundaries[1], 100)
    xv, yv = np.meshgrid(x, y)
    Xeval = np.vstack((xv.flatten(), yv.flatten())).T
    
    # query gp mean prediction
    Ypred = gp.predict_noiseless(Xeval)[0].reshape(100,100)
    
    # show colorbar
    colorbar = axis.contourf(xv,yv,Ypred, alpha=0.5)
    fig.colorbar(colorbar)
    
    # set boundaries
    axis.set_xlim(boundaries[0])
    axis.set_ylim(boundaries[1])


def plot_gp_3d(gp, boundaries, axis=None, fig=None):
    """
    Helper function for a 3d plot of a GP. The axis, if passed as argument, needs to use
    projection='3d'
    """
    
    if fig is None:
        fig = plt.gcf()
    if axis is None:
        axis = fig.gca(projection='3d')
    
    # plot data
    axis.scatter(gp.X[:,0], gp.X[:,1], gp.Y[:,0], marker='x', zorder=10)
    
    # create evaluation grid
    x = np.linspace(*boundaries[0], 100)
    y = np.linspace(*boundaries[1], 100)
    xv, yv = np.meshgrid(x, y)
    Xeval = np.vstack((xv.flatten(), yv.flatten())).T
    
    # query gp mean prediction
    Ypred = gp.predict_noiseless(Xeval)[0].reshape(100,100)
    
    # plot mesh
    axis.plot_wireframe(xv,yv,Ypred, alpha=0.3)
    
    # set boundaries
    axis.set_xlim(boundaries[0])
    axis.set_ylim(boundaries[1])

We will now define a 2d data set. For simplicity, we evaluate our accelerator dummy at `n=20` uniformly random points. As before, you can play around with then number of data points, the kernel and optimizing the hyperparameters.

In [ ]:
# domain boundaries
boundaries = [(-1,1), (-2,2)]

n = 20  # number of data points

# get some random evaluation points
X_2d = np.vstack((np.random.uniform(*boundaries[0], size=n),
               np.random.uniform(*boundaries[1], size=n))).T

# get data
Y_2d = np.array([[accelerator.get(x=x)] for x in X_2d])

# set up the gp model with a Matern52 kernel
matern52 = GPy.kern.Matern52(lengthscale=0.5, input_dim=2, variance=2)
gp_2d = GPy.models.GPRegression(X=X_2d, Y=Y_2d, kernel=matern52, noise_var=0.1)

# optionally, optimize hyperparameters
# gp_2d.optimize_restarts()

display(gp_2d)

# set up the plots
fig = plt.gcf()
fig.set_size_inches(10,3.5)
axis1 = plt.subplot(121, projection='3d')
axis2 = plt.subplot(122)

# 3d plot of posterior mean
plot_gp_3d(gp_2d, boundaries, axis=axis1)

# contour plot of posterior mean
plot_gp_contour(gp_2d, boundaries, axis=axis2)